In [1]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
import os
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import func

In [2]:
app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///../data/new_olympics.db'
db = SQLAlchemy(app)



C:\Users\norfe\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
sqlite3.connect('../data/olympics_raw.sqlite')
conn = sqlite3.connect('../data/new_olympics.db')

In [4]:
pd.read_sql('SELECT * FROM olympics_raw LIMIT 1', conn)

,id,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal,country_id
0,1,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold,1


In [5]:
 # Create engine using the `demographics.sqlite` database file
Base = automap_base()    
engine = create_engine("sqlite:///../data/new_olympics.db")
Base.prepare(engine, reflect=True)

In [6]:
Base.classes.keys()

['country_ref', 'olympics_raw']

In [7]:
Base.metadata

MetaData(bind=None)

In [8]:
class olympic_raw(db.Model):
    __tablename__ = 'olympics_raw'
    __table_args__ = {'extend_existing': True} 
    id = db.Column(db.Integer, primary_key=True)
    City = db.Column(db.String(64))
    Edition =  db.Column(db.String(64))
    Event= db.Column(db.String(64))
    Event_Gender= db.Column(db.String(64))
    Medal = db.Column(db.String(64))
    country_id =db.Column(db.Integer)
    NOC = db.Column(db.String(64))
    Gender = db.Column(db.String(64))
    Discipline = db.Column(db.String(64))
    Athlete = db.Column(db.String(64))

In [9]:
class raw(db.Model):
    __tablename__ = 'olympics_raw'
    __table_args__ = {'extend_existing': True} 
    id = db.Column(db.Integer, primary_key=True)
    City = db.Column(db.String(64))
    Edition =  db.Column(db.String(64))
    Event= db.Column(db.String(64))
    Event_Gender= db.Column(db.String(64))
    Medal = db.Column(db.String(64))
    country_id =db.Column(db.Integer, db.ForeignKey('country_ref.id'))
    NOC = db.Column(db.String(64))
    Gender = db.Column(db.String(64))
    Discipline = db.Column(db.String(64))
    Athlete = db.Column(db.String(64))
    Sport = db.Column(db.String(64))
    def __repr__(self):
        return '<raw %r>' % (self.id)

In [10]:
class country_ref(db.Model):
    __tablename__ = 'country_ref' 
    __table_args__ = {'extend_existing': True} 
    id = db.Column(db.Integer, primary_key=True)
    country_name = db.Column(db.String(64))
    code =db.Column(db.String(64))
    flag_source = db.Column(db.String(64))
    flag_image = db.Column(db.String(64))
    def __repr__(self):
        return '<country_ref %r>' % (self.id)


In [11]:
countries = Base.classes.country_ref


In [12]:
cr = db.session.query(country_ref.id, country_ref.code, country_ref.country_name,country_ref.flag_image).all()


In [18]:
count = db.session.query(raw.NOC,raw.Edition, raw.Sport, func.count(raw.id)).group_by(raw.Edition,raw.Sport).filter(raw.NOC == 'USA').all()
count

[('USA', '1896', 'Athletics', 17),
 ('USA', '1896', 'Shooting', 3),
 ('USA', '1900', 'Athletics', 39),
 ('USA', '1900', 'Cycling', 1),
 ('USA', '1900', 'Golf', 4),
 ('USA', '1900', 'Rowing', 9),
 ('USA', '1900', 'Sailing', 1),
 ('USA', '1900', 'Tennis', 1),
 ('USA', '1904', 'Aquatics', 28),
 ('USA', '1904', 'Archery', 30),
 ('USA', '1904', 'Athletics', 72),
 ('USA', '1904', 'Boxing', 19),
 ('USA', '1904', 'Cycling', 21),
 ('USA', '1904', 'Fencing', 6),
 ('USA', '1904', 'Football', 46),
 ('USA', '1904', 'Golf', 23),
 ('USA', '1904', 'Gymnastics', 44),
 ('USA', '1904', 'Lacrosse', 13),
 ('USA', '1904', 'Roque', 3),
 ('USA', '1904', 'Rowing', 36),
 ('USA', '1904', 'Tennis', 12),
 ('USA', '1904', 'Tug of War', 15),
 ('USA', '1904', 'Weightlifting', 5),
 ('USA', '1904', 'Wrestling', 21),
 ('USA', '1908', 'Aquatics', 6),
 ('USA', '1908', 'Archery', 1),
 ('USA', '1908', 'Athletics', 39),
 ('USA', '1908', 'Jeu de paume', 1),
 ('USA', '1908', 'Shooting', 14),
 ('USA', '1908', 'Wrestling', 2),
 

In [17]:
subq = (db.session.query(raw.country_id,raw.Medal,func.count(raw.id)).group_by(raw.country_id,raw.Medal)).filter(raw.Medal=='Gold').subquery()

In [ ]:
db.session.query(subq).all()

In [15]:
db.session.query(country_ref.country_name, country_ref.code, subq).join(subq).all()

NameError: name 'subq' is not defined

In [16]:
db.session.query(country_ref,count).filter(country_ref.id == count.country_id).all()

InvalidRequestError: SQL expression, column, or mapped entity expected - got '[('HUN', '1896', 'Aquatics', 11), ('USA', '1896', 'Athletics', 37), ('GBR', '1896', 'Cycling', 16), ('GRE', '1896', 'Fencing', 8), ('GER', '1896', 'Gymnastics', 45), ('GRE', '1896', 'Shooting', 15), ('ZZX', '1896', 'Tennis', 10), ('GRE', '1896', 'Weightlifting', 6), ('GRE', '1896', 'Wrestling', 3), ('HUN', '1900', 'Aquatics', 61), ('FRA', '1900', 'Archery', 18), ('AUS', '1900', 'Athletics', 76), ('ESP', '1900', 'Basque Pelota', 4), ('GBR', '1900', 'Cricket', 24), ('FRA', '1900', 'Croquet', 8), ('FRA', '1900', 'Cycling', 6), ('BEL', '1900', 'Equestrian', 9), ('FRA', '1900', 'Fencing', 21), ('BEL', '1900', 'Football', 35), ('GBR', '1900', 'Golf', 6), ('FRA', '1900', 'Gymnastics', 3), ('ZZX', '1900', 'Polo', 12), ('NED', '1900', 'Rowing', 68), ('FRA', '1900', 'Rugby', 47), ('FRA', '1900', 'Sailing', 26), ('FRA', '1900', 'Shooting', 52), ('FRA', '1900', 'Tennis', 24), ('ZZX', '1900', 'Tug of War', 12), ('GER', '1

In [ ]:
results=db.session.query(country_ref.country_name, country_ref.code, subq).join(subq).all()
results

In [ ]:
  country_dic ={}

In [ ]:
for result in results:
    if result[2] in country_dic:
        country_dic[result[2]]['medals'][result[3]] = result[4]
    else:
        country ={}

        country['name']  = result[0]
        country['code'] = result[1]
        country['medals'] = {}
        country['medals'][result[3]] = result[4]
        country_dic[result[2]] = country

In [32]:
db.session.query(raw.Sport,func.count(raw.id)).group_by(raw.Sport).filter(country_ref.code=='FRA').all()


[('Aquatics', 3828),
 ('Archery', 305),
 ('Athletics', 3448),
 ('Badminton', 120),
 ('Baseball', 335),
 ('Basketball', 940),
 ('Basque Pelota', 4),
 ('Boxing', 842),
 ('Canoe / Kayak', 1002),
 ('Cricket', 24),
 ('Croquet', 8),
 ('Cycling', 1025),
 ('Equestrian', 894),
 ('Fencing', 1547),
 ('Football', 1387),
 ('Golf', 30),
 ('Gymnastics', 2214),
 ('Handball', 886),
 ('Hockey', 1325),
 ('Ice Hockey', 27),
 ('Jeu de paume', 3),
 ('Judo', 435),
 ('Lacrosse', 59),
 ('Modern Pentathlon', 174),
 ('Polo', 66),
 ('Rackets', 10),
 ('Roque', 3),
 ('Rowing', 2523),
 ('Rugby', 192),
 ('Sailing', 1061),
 ('Shooting', 1105),
 ('Skating', 27),
 ('Softball', 180),
 ('Table Tennis', 120),
 ('Taekwondo', 80),
 ('Tennis', 272),
 ('Triathlon', 18),
 ('Tug of War', 94),
 ('Volleyball', 910),
 ('Water Motorsports', 5),
 ('Weightlifting', 548),
 ('Wrestling', 1140)]

In [33]:
data =db.session.query(country_ref.code, country_ref.country_name, country_ref.id,country_ref.flag_image).filter(country_ref.code == 'FRA').all()

In [39]:
country_data = db.session.query(country_ref.id, country_ref.code, country_ref.country_name,country_ref.flag_image).filter(country_ref.code== 'FRA').all()
  

In [71]:
 word_cloud = db.session.query(raw.Sport,func.count(raw.id).label('nMed')).group_by(raw.Sport).filter(raw.country_id== country_data[0].id)

In [41]:
words = []
for sport in word_cloud:
    sp ={}
    sp['word'] = sport.Sport
    

[('Aquatics', 89),
 ('Archery', 46),
 ('Athletics', 100),
 ('Basketball', 26),
 ('Basque Pelota', 2),
 ('Boxing', 19),
 ('Canoe / Kayak', 47),
 ('Cricket', 12),
 ('Croquet', 8),
 ('Cycling', 130),
 ('Equestrian', 64),
 ('Fencing', 286),
 ('Football', 30),
 ('Gymnastics', 51),
 ('Handball', 31),
 ('Judo', 37),
 ('Modern Pentathlon', 6),
 ('Rowing', 94),
 ('Rugby', 63),
 ('Sailing', 48),
 ('Shooting', 54),
 ('Table Tennis', 3),
 ('Taekwondo', 4),
 ('Tennis', 26),
 ('Tug of War', 6),
 ('Water Motorsports', 1),
 ('Weightlifting', 15),
 ('Wrestling', 16)]

In [58]:
help(word_cloud)

Help on list object:

class list(object)
 |  list() -> new empty list
 |  list(iterable) -> new list initialized from iterable's items
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __le__(self, value, /

In [75]:
word_cloud[0].nMed

89